In [ ]:
import glob
from datetime import date
from datetime import datetime
import stat
from datetime import timedelta
import random
import sys
import os

In [ ]:
fileNameDir = "/glade/scratch/tking/06"

In [ ]:
old = ["2000.txt", "2001.txt", "2002.txt", "2003.txt", "2004.txt"]
new = ["2000new.txt", "2001new.txt", "2002new.txt", "2003new.txt", "2004new.txt"]
days = [i for i in range(1, 366)]  # This was 363 originally
start = date(1979, 1, 1)
# start_times = [7665, 8030, 8395, 8760, 9125] # 21, 22, 23... years after 1979 = 2000, 2001, 2002, etc
start_times = [7669, 8035, 8400, 8765, 9130] # updated for leap years & index was 1 off
s = dict()
# leap years: 1980, 1984, 1988, 1992, 1996, 2000 (each of these has 366 days)


In [ ]:
# loop through years 2000-2004
for i in range(5):
    # open and read file fi, write to fw
    fi = open(fileNameDir + "/" + old[i], "r")
    fw = open(fileNameDir + "/" + new[i], "w+")
    fi_lines = fi.readlines()
    s[i] = set([])
    # each line has an index and date, add index to set s[i] if it is not yet in set
    for x in fi_lines:
        idx, fn = x.split()
        s[i].add(int(idx))
    fi.close()
    j = 0
    temp = []
    while j < 60: # why randomly assign 60 times that are not in s? to randomize order of indices? seems like indices change but order doesnt
        # while j < 365: # use to check all dates
        # there are 60 total lines...
        day = random.sample(days,1)[0]
        # day = days[j]
        hour = random.sample([1,2,3,4,5,6,7,8],1)[0]  # use the index corresponding to 3hourly files
        time = (day * 8) + hour   # time index is the number of days multiplied by 8 because it's 3hourly data
        if not (time in s[i]):
            temp.append((day, time, hour))
            j += 1
    temp.sort(key=lambda x: x[1])
    for d, t, h in temp:
        fname = (start + timedelta(days=start_times[i]+d)).strftime("data-%Y-%m-%d-{}-0.jpg".format(h))
        print("{} {}".format(t, fname))
        fw.write("{} {}\n".format(t, fname))
    fw.close()

In [ ]:
index = glob.glob('/global/project/projectdirs/ClimateNet/image_scripts/second200_filenames.txt')

In [ ]:
index

# Move old filenames to corrected filenames where index aligns with date

In [ ]:
old_filenames_qa1_antarctic = ["data-2001-01-01-00-2_0.nc",  "data-2001-05-14-00-2_0.nc",  "data-2001-09-08-00-2_0.nc",
    "data-2001-01-24-00-2_0.nc",  "data-2001-05-14-00-2_1.nc",  "data-2001-09-09-00-2_0.nc",
    "data-2001-01-24-00-2_1.nc",  "data-2001-05-28-00-2_0.nc",  "data-2001-09-26-00-2_0.nc",
    "data-2001-01-24-00-2_2.nc",  "data-2001-06-20-00-2_0.nc",  "data-2001-10-03-00-2_0.nc",
    "data-2001-02-14-00-2_0.nc",  "data-2001-06-20-00-2_1.nc",  "data-2001-10-06-00-2_0.nc",
    "data-2001-02-14-00-2_1.nc",  "data-2001-06-23-00-2_0.nc",  "data-2001-10-25-00-2_0.nc",
    "data-2001-02-17-00-2_0.nc",  "data-2001-06-23-00-2_1.nc",  "data-2001-10-25-00-2_1.nc",
    "data-2001-02-17-00-2_1.nc",  "data-2001-06-30-00-2_0.nc",  "data-2001-11-04-00-2_0.nc",
    "data-2001-02-20-00-2_0.nc",  "data-2001-06-30-00-2_1.nc",  "data-2001-11-08-00-2_0.nc",
    "data-2001-03-01-00-2_0.nc",  "data-2001-07-14-00-2_0.nc",  "data-2001-11-12-00-2_0.nc",
    "data-2001-03-01-00-2_1.nc",  "data-2001-07-14-00-2_1.nc",  "data-2001-11-22-00-2_0.nc",
    "data-2001-03-01-00-2_2.nc",  "data-2001-07-23-00-2_0.nc",  "data-2001-12-03-00-2_0.nc",
    "data-2001-03-01-00-2_3.nc",  "data-2001-07-23-00-2_1.nc",  "data-2001-12-05-00-2_0.nc",
    "data-2001-03-03-00-2_0.nc",  "data-2001-07-23-00-2_2.nc",  "data-2001-12-23-00-2_0.nc",
    "data-2001-03-03-00-2_1.nc",  "data-2001-07-23-00-2_3.nc",  "data-2001-12-23-00-2_1.nc",
    "data-2001-03-03-00-2_2.nc",  "data-2001-08-03-00-2_0.nc",  "data-2002-01-09-00-2_0.nc",
    "data-2001-03-12-00-2_0.nc",  "data-2001-08-03-00-2_1.nc",  "data-2002-01-10-00-2_0.nc",
    "data-2001-03-15-00-2_0.nc",  "data-2001-08-03-00-2_2.nc",  "data-2002-01-30-00-2_0.nc",
    "data-2001-03-15-00-2_1.nc",  "data-2001-08-14-00-2_0.nc",  "data-2002-02-07-00-2_0.nc",
    "data-2001-03-25-00-2_0.nc",  "data-2001-08-15-00-2_0.nc",  "data-2002-02-17-00-2_0.nc",
    "data-2001-03-25-00-2_1.nc",  "data-2001-08-21-00-2_0.nc",  "data-2002-02-22-00-2_0.nc",
    "data-2001-03-25-00-2_2.nc",  "data-2001-08-25-00-2_0.nc",  "data-2002-02-22-00-2_1.nc",
    "data-2001-04-05-00-2_0.nc",  "data-2001-09-07-00-2_0.nc"]

In [ ]:
# calculate what index would have (incorrectly) been
first_file = 'data-2003-01-15-02-0.jpg' # old_filenames_qa1_antarctic[0]
first_file_year = first_file.split("-")[1]
first_file_month = first_file.split("-")[2]
first_file_day = first_file.split("-")[3]

# [7670, 8036, 8401, 8766, 9131]
if first_file_year == '2000':
    start_time = 7669
if first_file_year == '2001':
    start_time = 8035
if first_file_year == '2002':
    start_time = 8400
if first_file_year == '2003':
    start_time = 8765
if first_file_year == '2004':
    start_time = 9130

time_index_used = int(first_file_day) * 8

print(first_file_day)
print("day, time: {}, {}".format(first_file_day, time_index_used))
time_index_used

print(start_time+int(first_file_day))
print(timedelta(days=start_time+int(first_file_day)))

fname = (date(1979, 1, 1) + timedelta(days=start_time+int(first_file_day))).strftime("data-%Y-%m-%d-02-0.jpg")
print(fname)

# maybe corrected dates are just off by 5 for 2000 and 6 for 2001+??? So, for year 2000, add 5 days, and for other years, add 6 days?
# but then indexing was off by one as well to return the same filetime, so actually +4 and +5


In [ ]:
# use index to calculate what date/time should have been


In [ ]:
# write out name of corrected file


In [ ]:
for i in range(len(old_filenames))
    print("mv {} {}".format(old_filenames[i], new_filenames[i]))